# Working with files in Python

We will cover only the necessary basics of reading and writing files in Python. You can find more details in, e.g., 
* https://docs.python.org/3/library/filesys.html
* https://realpython.com/working-with-files-in-python/


The cell below just prepares s temporary directory "tempdir" with a text file inside:

```
    .
    └── tempdir
        └── example.txt
```

Just run it, you do not need to understand the code inside now. 
After this lecture though, you will understand it much better.

In [ ]:
# set up a temp directory and file
import pathlib
import shutil

temp_dir = pathlib.Path("tempdir")
shutil.rmtree(temp_dir.name, ignore_errors=True)
temp_dir.mkdir(parents=True, exist_ok=True)
example_file = temp_dir / pathlib.Path("example.txt")
example_file.write_text("This is an example text file.\nIt has two lines.\n");

## Reading files

The standard pattern to open and read a file is using the `with` keyword, i.e. the [context manager type](https://docs.python.org/3/library/stdtypes.html#context-manager-types)
that we will not explain here. It is sufficient to remember that the `with` block here makes sure that the file is closed
in case any error happens, which is important because open files are operating system resources that are not infinite.

In [ ]:
file_path = "tempdir/example.txt"

with open(file_path, "r") as file_reader:
    file_text = file_reader.read()

print("The file contains this text:")
print(file_text)


Note that the object that contains a "handle" of the opened file is `file_reader`, i.e. the symbol that follows the `as` keyword. 
Also note that only the file processing part (reading) is inside the `with` block. 

There are two points that we should improve: It is not ideal to compose paths with directories using just plain `/` character, 
which on Windows may also be `\` in which case it becomes tricky as `\` is a special character in Python.
Also, often we do not / should not just read the whole contents of the file because the file may be too large or too slow to read at once
and also because we often can (and should) process files by parts (characters, lines, etc.). 
That's why the object returned by `open` is iterable; the iteration is line by line.

We can improve this by using `pathlib` for path operations and by iterating over the opened file instead of reading it all at once. 
Let's say we are interested in the first words of each line only.

In [ ]:
import pathlib

# we split paths into separate directories and file name
# the / operator (i.e. not charater) safely and correctly joins the path parts
# we can safely join Path objects with strings, yielding a Path object
directory_path = pathlib.Path("tempdir") 
file_path = directory_path / "example.txt" 

# .open method is used instead of the open function
with file_path.open("r") as file_reader:
    first_words = []
    # iterate over lines
    for line in file_reader:
        first_words.append(line.split(" ")[0])

print(first_words)

`file_reader` is an "IOSomething" instance: An Input / Output type that can read / write
from files, memory, etc.

In [ ]:
file_reader

And it is closed outside of the `with` block already:

In [ ]:
file_reader.read()

## Writing files

Creating and writing files is a straightforward modification of the reading pattern. 
We just need to open a file in a writeable mode 
and use the `write` (or `writelines` or some other) method.

In [ ]:
new_file_path = directory_path / "new_example.txt"

# the mode is "w" for writing
with new_file_path.open("w") as file_writer:
    for i, word in enumerate(first_words):
        file_writer.write(f"{i}. {word}\n")

print(f"{new_file_path.name} contains:")
# read_text is a shortcut to read the whole file contents
print(new_file_path.read_text())

## `pathlib` vs `os.path`

`pathlib` is a relatively new to Python (introduced in 3.5) and you will encounter the older "non-object"
approach many times. This approach uses paths in strings and 
functions from `os` and `os.path` modules instead of `pathlib.Path` objects.
If this is the approach to be used, remember to use functions like `os.path.join` for
joining directory and files names (instead of plain string manipulation) 
in order to keep your code safe and clean.

## Binary files

We have been working with text files up to know. Implicitly, we used the `UTF-8` encoding.
For binary files we need to use `b` in the `mode` argument of `open`. While operations on text
files are based on `str`, binary operations use the `bytes` type.